In [13]:
import zipfile
import os
import numpy as np
import cv2
from sklearn.utils import shuffle

# Zip dosyasını çıkarma işlemi, veriyi /content/CaltechTinyTwoClass klasörüne çıkarıyoruz
zip_path = '/content/CaltechTinyTwoClass.zip'
extract_path = '/content/CaltechTinyTwoClass'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Eğitim ve test verileri için dosya yollarını tanımlama
train_path = os.path.join(extract_path, 'CaltechTinyTwoClass', 'train')
test_path = os.path.join(extract_path, 'CaltechTinyTwoClass', 'test')

# Görüntü boyutu ve özellik sayısını tanımlıyoruz
image_size = (128, 128)  # Görüntülerin boyutu 128x128 piksel
n_features = 128 * 128 * 3  # 128x128 boyutunda ve 3 kanallı, bu nedenle toplam özellik sayısı 49152
n_classes = 2  # 2 farklı sınıf var

# Sınıf etiketlerini isimlendirme
class_map = {0: 'camera', 1: 'pizza'}
label_map = {'camera': 0, 'pizza': 1}

# Veriyi yükleme ve yeniden boyutlandırma fonksiyonu
def load_data(data_path):
    inputs, labels, file_paths = [], [], []

    # Her sınıfın klasörünü dolaşarak görüntüleri okuma
    for class_name, label in label_map.items():
        class_folder = os.path.join(data_path, class_name)
        for img_name in os.listdir(class_folder):
            img_path = os.path.join(class_folder, img_name)
            img = cv2.imread(img_path)  # Görüntüyü yükle
            if img is not None:
                img = cv2.resize(img, image_size)  # Görüntüyü 128x128 boyutuna yeniden boyutlandır
                inputs.append(img.flatten())  # 128x128x3 görüntüyü 1x49152 vektörüne dönüştür
                labels.append(label)  # Sınıf etiketini kaydet
                file_paths.append(img_path.replace(extract_path, ''))  # Görüntü dosya yolunu kaydet

    # Tüm piksel değerlerini 0-1 arasında olacak şekilde normalleştir
    inputs = np.array(inputs, dtype=np.float32) / 255.0
    labels = np.array(labels, dtype=np.int32)
    return inputs, labels, file_paths

# Eğitim verilerini yükle ve ön işleme
train_inputs, train_labels, _ = load_data(train_path)
train_inputs = np.hstack((train_inputs, np.ones((train_inputs.shape[0], 1))))  # Girişlere bias (1) ekleniyor (49152 + 1 = 49153 özellik)
train_inputs, train_labels = shuffle(train_inputs, train_labels)  # Eğitim verilerini karıştırıyoruz

# Softplus aktivasyon fonksiyonunu tanımlama
def softplus(x):
    # Taşmaları önlemek için x değerini -500 ile 500 arasında sınırlıyoruz
    x = np.clip(x, -500, 500)
    return np.log(1 + np.exp(x))  # Softplus aktivasyon fonksiyonu

# Softplus fonksiyonunun türevini tanımlama
def der_softplus(x):
    # Taşmaları önlemek için x değerini -500 ile 500 arasında sınırlıyoruz
    x = np.clip(x, -500, 500)
    return 1 / (1 + np.exp(-x))  # Softplus türevi döndürülüyor

# Perceptron eğitim fonksiyonu - eğitim süreci burada başlıyor
def trainPerceptron(inputs, t, weights, rho, iterNo):
    # iterasyon sayısına göre eğitim döngüsünü başlat
    for _ in range(iterNo):
        inputs, t = shuffle(inputs, t)  # Her iterasyonda veriyi karıştırıyoruz

        # Her bir eğitim örneğini sırayla işleme
        for i in range(inputs.shape[0]):
            x = inputs[i]  # Giriş vektörü (49153 özellik, bias dahil)
            target = t[i]  # Hedef etiket, -1 veya 1 olarak ayarlanacak

            # İleri besleme (feed-forward) adımı - ağın tahmini için ağırlıklar ve giriş vektörünü çarpıyoruz
            z = np.dot(weights, x)  # z: giriş ve ağırlıkların çarpımı sonucu (bias dahil)

            # Geri besleme (feed-backward) adımı - gradyan inişi ile ağırlık güncellemesi
            gradient = rho * (-target) * der_softplus(-target * z) * x  # Gradyan hesaplama
            weights -= gradient  # Ağırlıkları gradyan değeriyle güncelliyoruz

    return weights

# Tek model eğitmek için ağırlıkları başlatma (rastgele küçük değerlerle)
weights = np.random.randn(n_features + 1)  # 49153 ağırlık, bias dahil

# Eğitim parametreleri
rho = 0.0001  # Öğrenme oranı
iterNo = 1000  # 1000 iterasyon

# Sınıfa özgü hedef değerlerini ayarlıyoruz (1 ve -1 olarak)
binary_labels = np.where(train_labels == 1, 1, -1)

# Tek perceptron modeli eğitimi
weights = trainPerceptron(train_inputs, binary_labels, weights, rho, iterNo)

# Eğitim tamamlandıktan sonra ağırlıkları kaydet
np.save('weights.npy', weights)

# Perceptron test fonksiyonu
def testPerceptron(sample_test, weights):
    # Giriş verisi ile ağırlıkları çarparak skor hesaplama
    score = np.dot(weights, sample_test)  # 49153 özellik, bias dahil
    predicted_label = 1 if score > 0 else 0  # Pozitif skor pizza, negatif skor camera olarak sınıflandırılıyor
    predicted_class_name = class_map[predicted_label]  # Tahmin edilen sınıfın ismi
    return predicted_label, predicted_class_name

# Test verilerini yükle ve doğruluğu değerlendir
test_inputs, test_labels, test_file_paths = load_data(test_path)
test_inputs = np.hstack((test_inputs, np.ones((test_inputs.shape[0], 1))))  # Test verilerine bias ekleniyor (49152 + 1 = 49153 özellik)

# Kaydedilen ağırlıkları yükle
weights = np.load('weights.npy')

# Her örneği değerlendir
correct_predictions = 0
class_correct_predictions = {0: 0, 1: 0}
class_total_samples = {0: 0, 1: 0}

for i in range(test_inputs.shape[0]):
    # Tahmin edilen sınıf ve gerçek sınıfı karşılaştırarak doğruluğu kontrol ediyoruz
    predicted, predicted_class_name = testPerceptron(test_inputs[i], weights)
    actual = test_labels[i]
    actual_class_name = class_map[actual]
    class_total_samples[actual] += 1
    if predicted == actual:
        correct_predictions += 1
        class_correct_predictions[actual] += 1

    # Görüntü bazında tahmin edilen ve gerçek sınıf isimlerini yazdırıyoruz
    print(f'{test_file_paths[i]} - Gerçek Sınıf: {actual_class_name} ({actual}), Tahmin Edilen Sınıf: {predicted_class_name} ({predicted})')

# Genel doğruluğu hesaplayıp göster
accuracy = correct_predictions / test_inputs.shape[0]
print(f'Genel Doğruluk: {accuracy * 100:.2f}%')

# Her sınıfın doğruluğunu hesaplayıp göster
for class_label in class_correct_predictions:
    class_accuracy = (class_correct_predictions[class_label] / class_total_samples[class_label]) * 100
    class_name = class_map[class_label]
    print(f'{class_name} Sınıfı Doğruluğu: {class_accuracy:.2f}%')


/CaltechTinyTwoClass/test/camera/image_0046.jpg - Gerçek Sınıf: camera (0), Tahmin Edilen Sınıf: camera (0)
/CaltechTinyTwoClass/test/camera/image_0035.jpg - Gerçek Sınıf: camera (0), Tahmin Edilen Sınıf: pizza (1)
/CaltechTinyTwoClass/test/camera/image_0040.jpg - Gerçek Sınıf: camera (0), Tahmin Edilen Sınıf: camera (0)
/CaltechTinyTwoClass/test/camera/image_0037.jpg - Gerçek Sınıf: camera (0), Tahmin Edilen Sınıf: camera (0)
/CaltechTinyTwoClass/test/camera/image_0024.jpg - Gerçek Sınıf: camera (0), Tahmin Edilen Sınıf: camera (0)
/CaltechTinyTwoClass/test/pizza/image_0046.jpg - Gerçek Sınıf: pizza (1), Tahmin Edilen Sınıf: pizza (1)
/CaltechTinyTwoClass/test/pizza/image_0035.jpg - Gerçek Sınıf: pizza (1), Tahmin Edilen Sınıf: pizza (1)
/CaltechTinyTwoClass/test/pizza/image_0040.jpg - Gerçek Sınıf: pizza (1), Tahmin Edilen Sınıf: pizza (1)
/CaltechTinyTwoClass/test/pizza/image_0038.jpg - Gerçek Sınıf: pizza (1), Tahmin Edilen Sınıf: pizza (1)
/CaltechTinyTwoClass/test/pizza/image_003